In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Modeling libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Interpretability
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("=" * 80)
print("TASK 4: PREDICTIVE MODELING FOR RISK-BASED PRICING")
print("=" * 80)

# Load the processed data
try:
    df = pd.read_csv('../data/processed/processed_data.csv')
    print(f"Data loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print("ERROR: Processed data not found. Please run Task 1 first.")
    exit()

# --- Data Preparation for Modeling ---
print("\n" + "=" * 80)
print("DATA PREPARATION")
print("=" * 80)

# Create target variables
df['ClaimOccurred'] = (df['TotalClaims'] > 0).astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

print(f"Original dataset shape: {df.shape}")

# Feature Selection
# Remove columns with too many missing values or irrelevant for prediction
drop_cols = [
    'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 
    'VehicleIntroDate', 'YearMonth', 'LossRatio',
    'NumberOfVehiclesInFleet', 'CrossBorder',  # Too many missing
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank',  # Too many categories/missing
    'AccountType', 'MaritalStatus', 'Country', 'MainCrestaZone',
    'SubCrestaZone', 'ItemType', 'mmcode', 'make', 'Model',
    'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice',
    'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt',
    'Converted', 'TermFrequency', 'ExcessSelected', 'CoverCategory',
    'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
    'StatutoryRiskType'
]

# Keep only columns that exist
existing_drop_cols = [col for col in drop_cols if col in df.columns]
df_model = df.drop(columns=existing_drop_cols)

print(f"After dropping irrelevant columns: {df_model.shape}")

# Handle missing values
missing_before = df_model.isnull().sum().sum()
print(f"Missing values before cleaning: {missing_before}")

# Impute numerical columns with median
numerical_cols = df_model.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['TotalClaims', 'TotalPremium', 'ClaimOccurred', 'Margin']]

for col in numerical_cols:
    if df_model[col].isnull().sum() > 0:
        df_model[col] = df_model[col].fillna(df_model[col].median())

# Impute categorical columns with mode
categorical_cols = df_model.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    if df_model[col].isnull().sum() > 0:
        df_model[col] = df_model[col].fillna(df_model[col].mode()[0] if not df_model[col].mode().empty else 'Unknown')

missing_after = df_model.isnull().sum().sum()
print(f"Missing values after cleaning: {missing_after}")

# Feature Engineering
print("\n--- Feature Engineering ---")

# 1. Vehicle Age
current_year = 2015  # Based on data timeframe
df_model['VehicleAge'] = current_year - df_model['RegistrationYear']
df_model['VehicleAge'] = df_model['VehicleAge'].clip(lower=0, upper=50)  # Cap at 50 years

# 2. Premium to SumInsured Ratio
df_model['PremiumToSumInsured'] = df_model['CalculatedPremiumPerTerm'] / (df_model['SumInsured'] + 1)

# 3. Binary features from categorical
df_model['HasAlarm'] = df_model['AlarmImmobiliser'].notnull().astype(int)
df_model['HasTracking'] = df_model['TrackingDevice'].notnull().astype(int)

# 4. Log transform for skewed monetary variables
for col in ['TotalPremium', 'TotalClaims', 'SumInsured', 'CalculatedPremiumPerTerm', 'Margin']:
    if col in df_model.columns:
        df_model[f'Log_{col}'] = np.log1p(df_model[col].clip(lower=0))

print(f"After feature engineering: {df_model.shape}")

# --- MODEL 1: Claim Severity Prediction ---
print("\n" + "=" * 80)
print("MODEL 1: CLAIM SEVERITY PREDICTION")
print("=" * 80)
print("Target: TotalClaims (only for policies with claims > 0)")
print("Goal: Predict claim amount given a claim occurred")

# Filter for claims only
df_severity = df_model[df_model['TotalClaims'] > 0].copy()
print(f"Severity dataset: {len(df_severity)} claims")

# Define features and target for severity
severity_features = [
    'SumInsured', 'CalculatedPremiumPerTerm', 'VehicleAge',
    'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts',
    'NumberOfDoors', 'VehicleType', 'bodytype', 'Province', 'Gender',
    'PremiumToSumInsured', 'HasAlarm', 'HasTracking', 'PostalCode'
]

# Keep only existing features
severity_features = [col for col in severity_features if col in df_severity.columns]

X_sev = df_severity[severity_features]
y_sev = df_severity['TotalClaims']

print(f"Severity features: {len(severity_features)}")
print(f"Target range: R{y_sev.min():,.0f} to R{y_sev.max():,.0f}")
print(f"Mean claim amount: R{y_sev.mean():,.0f}")

# Split data
X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(
    X_sev, y_sev, test_size=0.2, random_state=42
)

print(f"\nTrain size: {len(X_train_sev):,}")
print(f"Test size: {len(X_test_sev):,}")

# Define preprocessing
categorical_features_sev = X_sev.select_dtypes(include=['object']).columns.tolist()
numerical_features_sev = X_sev.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_sev = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numerical_features_sev),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features_sev)
    ])

# Define models
severity_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, max_depth=5, random_state=42, verbosity=0)
}

# Train and evaluate models
severity_results = []
best_severity_model = None
best_severity_score = float('inf')

print("\n" + "-" * 80)
print("Training Severity Models...")
print("-" * 80)

for name, model in severity_models.items():
    print(f"\nTraining {name}...")
    
    # Create pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor_sev),
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    # Train
    pipeline.fit(X_train_sev, y_train_sev)
    
    # Predict
    y_pred = pipeline.predict(X_test_sev)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test_sev, y_pred))
    mae = mean_absolute_error(y_test_sev, y_pred)
    r2 = r2_score(y_test_sev, y_pred)
    
    print(f"  RMSE: R{rmse:,.2f}")
    print(f"  MAE: R{mae:,.2f}")
    print(f"  R²: {r2:.4f}")
    
    # Store results
    result = {
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    severity_results.append(result)
    
    # Track best model
    if rmse < best_severity_score:
        best_severity_score = rmse
        best_severity_model = pipeline

# Display severity results
print("\n" + "=" * 80)
print("SEVERITY MODEL COMPARISON")
print("=" * 80)

severity_df = pd.DataFrame(severity_results).sort_values('RMSE')
print(severity_df.to_string(index=False))

# --- MODEL 2: Claim Probability Prediction ---
print("\n" + "=" * 80)
print("MODEL 2: CLAIM PROBABILITY PREDICTION")
print("=" * 80)
print("Target: ClaimOccurred (binary: 0 = no claim, 1 = claim)")
print("Goal: Predict probability of a claim occurring")

# Use entire dataset for claim probability
df_claim_prob = df_model.copy()

# Define features for claim probability (can include more features)
claim_prob_features = severity_features + ['VehicleAge', 'PremiumToSumInsured', 'HasAlarm', 'HasTracking']

# Keep only existing features
claim_prob_features = [col for col in claim_prob_features if col in df_claim_prob.columns]

X_prob = df_claim_prob[claim_prob_features]
y_prob = df_claim_prob['ClaimOccurred']

print(f"Claim probability dataset: {len(X_prob):,} policies")
print(f"Claim rate: {y_prob.mean():.4f} ({y_prob.sum():,} claims)")
print(f"Features: {len(claim_prob_features)}")

# Split data
X_train_prob, X_test_prob, y_train_prob, y_test_prob = train_test_split(
    X_prob, y_prob, test_size=0.2, random_state=42, stratify=y_prob
)

print(f"\nTrain size: {len(X_train_prob):,}")
print(f"Test size: {len(X_test_prob):,}")

# Define preprocessing for probability model
categorical_features_prob = X_prob.select_dtypes(include=['object']).columns.tolist()
numerical_features_prob = X_prob.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_prob = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numerical_features_prob),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features_prob)
    ])

# Define classification models
classification_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest Classifier': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting Classifier': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost Classifier': XGBClassifier(n_estimators=100, max_depth=5, random_state=42, verbosity=0)
}

# Train and evaluate classification models
classification_results = []
best_classification_model = None
best_classification_score = 0

print("\n" + "-" * 80)
print("Training Classification Models...")
print("-" * 80)

for name, model in classification_models.items():
    print(f"\nTraining {name}...")
    
    # Create pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor_prob),
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    # Train
    pipeline.fit(X_train_prob, y_train_prob)
    
    # Predict
    y_pred = pipeline.predict(X_test_prob)
    y_pred_proba = pipeline.predict_proba(X_test_prob)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test_prob, y_pred)
    precision = precision_score(y_test_prob, y_pred)
    recall = recall_score(y_test_prob, y_pred)
    f1 = f1_score(y_test_prob, y_pred)
    roc_auc = roc_auc_score(y_test_prob, y_pred_proba)
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  ROC-AUC: {roc_auc:.4f}")
    
    # Store results
    result = {
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }
    classification_results.append(result)
    
    # Track best model (by F1-score)
    if f1 > best_classification_score:
        best_classification_score = f1
        best_classification_model = pipeline

# Display classification results
print("\n" + "=" * 80)
print("CLASSIFICATION MODEL COMPARISON")
print("=" * 80)

classification_df = pd.DataFrame(classification_results).sort_values('F1-Score', ascending=False)
print(classification_df.to_string(index=False))

# --- MODEL 3: Risk-Based Premium Calculation ---
print("\n" + "=" * 80)
print("MODEL 3: RISK-BASED PREMIUM CALCULATION")
print("=" * 80)
print("Concept: Premium = (Predicted Probability × Predicted Severity) + Costs + Profit Margin")

# Use best models to calculate risk-based premium
print("\nCalculating risk-based premiums for test set...")

# Get claim probabilities
y_prob_pred = best_classification_model.predict_proba(X_test_prob)[:, 1]

# Get severity predictions (need to